In [ ]:
#Determine whether the sequence contains non-ATGC characters.
def contains_invalid_chars(sequence):

    valid_chars = ['A','T','G','C']
    for char in sequence:
        if char not in valid_chars:
            print(char)
            return True
    return False

In [ ]:
#removing all records from genomes where any derived sequences contained inaccuracies
import pandas as pd
download = pd.read_excel(r'C:\Users\Qi\Desktop\E. coli_TnpB\Fig.1\IS605s\download\tnpB_loci_sequence_download.xlsx', sheet_name='Sheet1')

to_delete=[]
for index, row in download.iterrows():
    if contains_invalid_chars(row['sequence']) == True:
        to_delete.append(row['acc_num'])
print(to_delete)

# Filter the DataFrame to remove rows where acc_num is in the to_delete liindexst
clean_data = download[~download['acc_num'].isin(to_delete)]
print(clean_data)

In [ ]:
#Calculate the copy number of acc_num.
acc_counts = clean_data['acc_num'].value_counts()

acc_counts_df = acc_counts.reset_index()
acc_counts_df.columns = ['acc_num', 'copynumber']
print(acc_counts_df)
print('__________')
clean_data=pd.merge(left=clean_data, right=acc_counts_df, left_on='acc_num', right_on='acc_num',how='left')
clean_data['acc_num'] = clean_data.apply(lambda row: f"{row['acc_num']}_{row.name}", axis=1)
print(clean_data)
  

In [ ]:
#Print the downloaded sequences and save them as a file named clean_tnpB_loci.fast for use as a local BLAST database.
for index, row in clean_data.iterrows():
    print(f">{row['acc_num']}")
    print(row['sequence'])

In [ ]:
#Buide the local BLAST for clean_tnpB_loci
import subprocess
command = "makeblastdb -in clean_tnpB_loci.fasta -dbtype nucl"
subprocess.run(command, shell=True)

In [ ]:
#Determine if tnpA is intact
from Bio.Blast import NCBIXML
from Bio.Seq import Seq
import re

# Create an empty list to store DataFrames

TnpA_stutas=[]
with open(r'C:\Users\Qi\Desktop\E. coli_TnpB\Fig.1\IS605s\upload\bl21_TnpA_nucl_vs_clean_tnpB_loci.xml') as result_handle:
    blast_record = NCBIXML.read(result_handle)
    n=0

    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            ID = alignment.title.split(" ", 1)[1]
            sbjct_clean = re.sub(r'[^ATGC]', '', hsp.sbjct)
            TnpA_pro = Seq(sbjct_clean).reverse_complement().translate()
            if hsp.sbjct_end < hsp.sbjct_start:
                print("The tnpB and tnpA are in same direction")
            if len(TnpA_pro) > 135 and "*" not in str(TnpA_pro):
                TnpA_ORF='intact'
            else:
                TnpA_ORF='incomplete'
            TnpA_stutas.append({
                    'ID': ID,
                    'TnpA_pro_seq':str(TnpA_pro),
                    'TnpA_start':hsp.sbjct_start,
                    'TnpA_end':hsp.sbjct_end,
                    'TnpA_ORF':TnpA_ORF
            })
            
    df_TnpA=pd.DataFrame(TnpA_stutas)

tnpB_loci_TnpA=pd.merge(left=clean_data,right=df_TnpA,left_on='acc_num',right_on='ID',how='left')
print(tnpB_loci_TnpA)

In [ ]:
#Align the TTTAT TAM followed by the TAM+LE sequence of IS605 element from E. coli BL21 (DE3) against clean_tnpB_loci.
import subprocess
command = "blastn -query TAM_Le_seq.fasta -db clean_tnpB_loci.fasta -evalue 0.01 -num_threads 4 -out TAM_Le_seq_vs_upstream_tnpA.xml -max_target_seqs 6000 -max_hsps 1 -outfmt 5"
subprocess.run(command, shell=True)

In [ ]:
#Count the types of TAMs
from collections import Counter

# Create an empty list to store DataFrames

TAM=[]
with open(r'C:\Users\Qi\Desktop\E. coli_TnpB\Fig.1\IS605s\upload\TAM_Le_seq_vs_upstream_tnpA.xml') as result_handle:
    blast_record = NCBIXML.read(result_handle)
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            ID=alignment.title.split(" ", 1)[1]
            TAM.append({
                    'ID': ID,
                    'TAM':tnpB_loci_TnpA[tnpB_loci_TnpA['acc_num'] == ID]['sequence'].values[0][hsp.sbjct_start-hsp.query_start:hsp.sbjct_start-hsp.query_start+5]
            })
            #TAM.append(IS605_TnpA[IS605_TnpA['acc_num'] == ID]['genome_sequence'].values[0][hsp.sbjct_start-hsp.query_start-5:hsp.sbjct_start-hsp.query_start])
    df_TAM=pd.DataFrame(TAM)
    print(df_TAM)
tnpB_loci_TnpA_TAM=pd.merge(left=tnpB_loci_TnpA,right=df_TAM,left_on='acc_num',right_on='ID',how='left')
tnpB_loci_TnpA_TAM.to_excel(r'C:\Users\Qi\Desktop\E. coli_TnpB\Fig.1\IS605s\upload\tnpB_loci_TnpA_TAM.xlsx', index=False) 
tam_counts = Counter([entry['TAM'] for entry in TAM])
print(tam_counts)